# 5. 너의 속이 궁금해 - Class Activation Map 살펴보기

지금까지 딥러닝 모델을 기반으로 image classification, detection하는 여러가지 모델을 공부했다. 이미지를 모델의 입력에 맞게 전처리해주고, 모델에 통과시킨 뒤 결과를 해석해서 추론값을 얻어내는 과정을 했다.

개와 고양이를 분류하는 딥러닝 모델을 만들었다고 예를 들어보자. 모델이 이미지의 어느 부분을 보고 개라는 결론을 내렸을지 알 수 있을까? 그동안 우리가 다룬 딥러닝은 모델의 추론 근거를 알 수 없는 black box 모델이었다. 그래서 신뢰할 수 있는지조차 명확하지 않았다. 그래서 모델 추론의 신뢰성에 대한 답을 찾는 __XAI(Explainable Artficial Intelligence)__분야에 대해 알아보자. 이 기법은 모델의 성능을 개선할 수 있는 단서로도 유용하게 활용된다.

# 5-2. Explainable AI

많은 AI 모델들이 다양한 분야에 적용되어 좋은 성과를 내고 있다. 하지만 현실 문제에 신경망 모델을 적용할 때, 단순히 모델의 가중치를 최적화하는 과정 = 모델이 올바르게 동작한다 는 아니다.

모델이 잘못된 답변을 준다면 어떻게 개선할 수 있는지, 잘 동작한다면 왜 이런 선택을 했는지 알 수 있는게 __XAI(explainable AI)이다.   
[설명해주는 딥러닝 XAI](https://youtu.be/U43fxbC-4JQ)

### Image Classfication
![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-2.max-800x600.png)

위 그림은 분류 모델에 이미지가 입력으로 들어오는 상황이다. 일반적인 이미지 분류 모델은 이미지의 local feature를 추출하기 위해 CNN으로 구성된 특성 추출용 nackbone network가 앞에 있다. 여기서 추출된 feature map을 FC layer에 통과시켜 얻어진 logit을 softmax activation function에 통과시키면, 입력 이미지가 각 클래스에 속할 확률을 얻을 수 있다.   
[한 페이지 머신러닝-logit, sigmoid, softmax의 관계](https://opentutorials.org/module/3653/22995)

logit은 sigmoid의 역함수이며, softmax는 sigmoid를 K개의 클래스로 일반화한것이다.   
$t=logit(y)=ln(\frac{y}{1-y})$   
$y=sigmoid(t)=\frac{1}{1+exp(t)}$

이미지 분류 모델에서 얻은 logit 값이 K개 클래스 중 하나가 정답이라 가리키면, 그렇게 분류할 수 있는 근거를 이미지 분류 모델에서 찾아볼 수 있다.

# 5-3. CAM: Class Activation Map

CAM은 Class Activation Map을 뜻한다. __모델이 어떤 곳을 보고 어떤 클래스임을 짐작하고 있는지__ 확인할 수 있는 map아라고 보면 된다.

[Learning Deep Features for Discriminative Localization](https://arxiv.org/abs/1512.04150) 논문을 통해 확인할 수 있다.   
[CNN looking](https://youtu.be/fZvOy0VXWAI) 영상을 보면 매우 직관적이게 Activation map을 사용해서 얻은 결과를 알려주고 있다.

CAM 논문의 연구진은 결과를 얻기 위해 GAP(Global average pooling) 기법을 사용했다. 이를 통해 어떤 클래스가 어느 영역에서 활성화되었는지 알 수 있다.

### 1) GAP(Global Average Pooling)
- [Network In Network - 논문](https://arxiv.org/abs/1312.4400)
- [navisphere.net의 Network In Network리뷰](http://www.navisphere.net/5493/network-in-network/)
- [C4W1L09 Pooling Layers](https://youtu.be/8oOgPUO-TBY)

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-4.max-800x600.png)

일반적인 이미지 분류는 여러 CNN 레이어를 거쳐 feature를 추출하고, 그 feature map을 flattening 한 후 FC layer에 입력해 각 클래스에 따른 logit을 구하고 최종적으로 활성화 함수를 거친다.

그러나 Networ in Network 논문에서는 CNN 이후 FC layer 대신 GAP 연산을 한다. GAP은 매 채널별 average pooling을 채널 값 전체에 global하게 적용한다.   
위 그림을 예로 들면 6x6 크기의 채널 3개가 GAP를 수행하면서 채널별 한 개의 숫자로 요약되어 1x1 크기 채널 3개 벡터를 얻게된다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-3.max-800x600.png)

GAP 연산을 하고 그 결과에 softmax activation function을 적용한다. 이때 마지막 CNN 레이어의 채널 수는 데이터의 클래스 수에 맞춰 각 클래스에 따른 확률을 얻을 수 있도록 한다.   
논문에서 이렇게 적용하면 특성 맵의 각 채널이 클래스별 신뢰도를 나타내게 되어 해석이 쉽고, FC 레이어와 달리 최적화할 파라미터가 존재하지 않아 overfitting을 방지할 수 있다고 한다.

### 2) CAM
- [Learning Deep Features for Discriminative Localization](http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf)
- [[논문리뷰] - Learning Deep Features for Discriminative Localization, CVPR 2016](https://joungheekim.github.io/2020/09/29/paper-review/)

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-5.max-800x600.png)

일반적인 CNN은 커널 윈도우에 따라서 특성을 추출하므로 CNN 레이어를 가진 특성 맵에도 입력값의 위치정보가 유지된다. 따라서 특성 맵의 정보를 image detection이나 segmentation 등의 문제를 푸는 데 이용하기도 한다.

CAM을 얻을 수 있는 네트워크는 위 그림에서 볼 수 있다. CNN 레이어를 거쳐 뽑은 feature map에 GAP를 적용한다. 그리고 여기에 softmax 활성화 함수를 갖고 bias가 없는 FC layer가 적용된다.

CNN을 거친 feature map에 각 클래스에 대한 정보의 결과값이 여러 채널에 걸쳐 나타난다. GAP로 채널별 정보를 요악하면 softmax layer는 이 정보를 보고 각 클래스에 대한 개별 채널의 중요도를 결정한다.   
이렇게 softmaxlayer로 각 채널의 가중합을 구해 각 클래스가 활성화 맵의 어떤 부분을 주로 활성화시키는지 확인할 수 있다.

이렇게 얻은 feature map은 CNN의 출력값 크기와 같게 된다. 이를 interpolation(보간)을 통해 적절하게 원본 이미지 크기로 확대해주면 위와 같은 CAM을 얻을 수 있다.

수식으로 살펴보자.   

$S_c=\displaystyle\sum_{k}w^c_k\sum_{x,y}=\sum_{x,y}\sum_kw^c_kf_k(x,y)$

$k=1,2,...,n$인 $k$번쨰 채널에 $w^c_k$는 각 클래스 $(c)$노드와 $k$번째 채널 사이의 가중치 값이다. $f_k(x,y)$는 $k$번쨰 채널의 $x,y$ 요소의 활성화 값이다. 위 그림에서 파란색 네모 박스로 시각화 되어 있다.

이 두 가지를 곱하고 모든 채널과 $x, y$축에 대해 더하여 클래스별 점수 $S_c$를 구할 수 있다.

$M_c(x,y)=\displaystyle\sum_{k}w^c_kf_k(x,y)$

CAM을 얻기 위해 위처럼 모든 위치의 활성화를 더해줘 클래스별 점수를 얻은것과 달리 $x, y$ 위치에서 $k$개의 채널만 더해주어 위치 정보가 남도록 한다.   
이렇게 얻은 CAM은 각 위치에서 채널별 활성화 정도의 가중 합인 $M_c(x,y)$가 된다. 이는 모델이 클래스 $c$에 대해 각 위치를 얼마나 보고있는지 나타낸다.

# 5-4. Grad-CAM

[Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization](https://arxiv.org/abs/1610.02391)

위의 CAM은 활성화 맵을 얻기 위해 GAP을 사용해 $f_k$를 구하고, 그 뒤로 FC layer $w^c_k$를 추가로 붙여야 한다. 또 가장 마지막 CNN 레이어의 결과물만 시각화할 수 있다.

Grad-CAM(Gradient CAM)은 CAM 모델의 구조가 제한되는 문제를 해결하고, 다양한 모델의 구조를 해석할 수 있는 방법을 제안한다. Grad-CAM을 사용하면 CNN 기반의 네트워크는 굳이 모델 구조를 변경할 필요도 없고, 분류 문제 외에 다른 task들에 유연하게 대처할 수 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-8.max-800x600.png)

위 이미지는 개와 고양이가 있는 이미지에 대해 Guided Backprop, Grad-CAM, Occlussion map의 시각화를 비교하고 있다. Grad-CAM은 높은 분별력과 큰 dimension을 갖는 CAM을 만드는것이 중요하게 보인다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-9.max-800x600.png)

Grad-CAM의 전체적인 전개도이다. 왼쪽에서 모델의 구조를 볼 수있다. 이미지 입력으로 CNN을 거쳐 feature map을 추출하고 그 뒤로 task에 따라 다양한 레이어들이 사용된다.

우측의 Image Classification, image captioning, visual question answering은 Grad-CAM이 적용될 수 있는 다양한 CV 문제들을 설명한다. 

image classification은 이미지가 무엇인지 분류를 하는 task이고, image captioning은 이미지에 대한 설명을 만드는 task, visual question answering은 VQA라고도 불리며, 질문과 이미지가 주어졌을 때 이에 대한 답변을 내는 task이다.

이렇게 복잡한 모델들은 다양한 모델이 동시에 사용되므로 설명하기 매우 어렵다. Grad-CAM은 복잡한 태스크에 사용되는 모델에서도 사용될 수 있다는 점을 장점으로 내세운다.

### gradient를 통한 Weight Score 계산
CAM에서 softmax를 가진 FC layer의 가중치를 통해 어떤 클래스에 대한 각 채널의 중요도 또는 가중치를 얻어낸다. 그렇가면 Grad-CAM에서는 어떻게 CAM을 얻어낼까?   
gradient를 이용한다. 원하는 클래스에 대해 관찰하는 레이어로 들어오는 gradient를 구할 수 있다면, 해당 클래스를 활성화할 때, 레이어의 특성 맵에서 어떤 채널이 중요하게 작용하는지 알 수 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GC-3-L-10.png)

backpropagation을 통해 얻은 $k$번째 채널의 gradient를 통해 weight score를 구하는 식이다.

$y$는 모델의 출력값이고, $A$는 activation map을 의미한다. $i,j$는 각각 x,y축이고, $Z$는 전체 map의 크기이다. 

해당 식을 통해 $k$개의 채널을 가진 activation map에서 각 채널이 어떤 클래스를 활성화하는데 얼마나 중요하게 작용하는지 가중치 점수를 구할 수 있다. __가중치를 구하기 위해 CAM처럼 별도의 weight 파라미터를 도입할 필요가 없다.__

이 정보를 합쳐 활성화 맵에서 어떤 클래스의 위치에 따른 활성화를 보려면 어떻게 할까? Grad-CAM은 이를 위해 $k$번째 활성화 맵과 이 가중치를 곱하여 합한뒤 ReLU 활성화 함수를 통해 클래스에 따른 Grad-CAM을 얻는다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GC-3-L-11.png)   

# 5-5. ACoL:Adversarial Complementary Learning

[Adversarial Complementary Learning for Weakly Supervised Object Localization](https://openaccess.thecvf.com/content_cvpr_2018/papers/Zhang_Adversarial_Complementary_Learning_CVPR_2018_paper.pdf)

CAM의 신기한 점은 클래스와 이미지 데이터로 학습을 하는데 위치 정보까지 얻을 수 있는 것이다. 이렇게 직접적으로 정답 위치 정보를 주지 않아도 간접적인 정보를 활용하여 원하는 정보를 얻어낼 수 있도록 학습하는 방식의 모델을 __약지도학습(weakly supervised learning)__이라고 한다.   
지금까지 본 CAM, Grad-CAM 그리고 지금 보는 ACoL은 약지도학습 기법을 활용해 object detection을 수행할 수 있다.

### 1) 약지도학습(weakly supervised learning)
딥러닝을 학습하면서 약지도학습과 준지도학습이라는 단어를 볼 수 있다. 두 개념이 다소 혼용되어 논문 [A brief introduction to weakly supervised learning](https://academic.oup.com/nsr/article/5/1/44/4093912)에서 정의한것을 토대로 나누어보자.

- incomplete supervision   
학습 데이터 중 일부에만 라벨이 달린 경우, 일반적으로 말하는 준지도학습과 같은 경우(개, 고양이 분류 10000개의 이미지 중 1000개만 라벨이 있는 경우)
- inexact supervision   
학습 데이터의 라벨이 충분히 정확하게 달려있지 않은 경우 (개, 고양이를 object detection 또는 semantic segmentation 해야하지만 이미지에 정확한 bounding box는 주어져 있지 않고 이미지가 개인지 고양이인지 정보만 라벨로 달려있는 경우)
- inaccurate supervision   
학습 데이터에 Noise가 있는 경우(개, 고양이 라벨링 잘못됨)

약지도학습은 위 3가지 경우를 포괄적으로 사용되곤 하지만, 오늘 우리가 다루고자 하는 것은 inexact supervision에 해당하는 경우이다.

일반적으로 image classification용 학습데이터보다 bounding box 정보까지 정확하게 포함해야하는 object detection, semantic segmentation을 위한 학습 데이터가 제작비용이 훨씬 많이 든다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/grad-cam-counterfactual.png)

Grad-CAM을 통한 counterfacutual explanation 예시이다. Grad-CAM을 통해서 개와 고양이의 특징이 두드러지게 하는 영역의 가중치 점수를 계산할 수 있었지만, 해당 가중치 점수를 제거해주면 image classification 모델에서 해당 클래스에 대한 prediction이 바뀌게 될 수도 있다.   
가중치 점수를 제거했을 때 prediction이 바뀌도록 하는 가중치 영역을 모으면 bounding box 라벨을 한 번도 보지 않고 object detection을 해낼 수 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/grad-cam_segmentation.png)

위와 같은 논리로 Grad-CAM을 이용해 semantic segmentation도 수행할 수 있다.

또 다른 예로 [네이버 랩스의 이미지기반의 차선변경 알고리즘(SLC)은 무엇을 보면서 판단을 할까?](https://www.naverlabs.com/storyDetail/16) 도 있다.

### 2) Averarial Complementary Learning
![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-13.max-800x600.png)

Adversarial은 적대적인이다. GAN에서도 이러한 방식을 사용한다. 논문 제목에서 사용된 ACoL(Adversarial complementary Learning)은 어떤 방법일까?   
ACoL은 모델의 학습에는 끝단이 두 branch로 나뉜다. CAM을 만들기 위해 활용한 특성 맵을 두 가지로 분리한 것이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-14.max-800x600.png)

CAM과 Grad-CAM은 weakly supervised 방식의 object detection, segmenation의 가능성을 보여주었다. 그러나 한 가지 문제가 있다. CAM을 통해 본 활성화 맵을 보면 가장자리보다 특징이 주로 나타나는 위치에 중점적으로 활성화되는 모습을 볼 수 있다.

하지만 object detection은 새의 부리나 동물의 눈, 시계의 숫자와 같이 부분적 특성이 아닌 물체의 전체적인 형태와 윤곽을 정확하게 구분해 내는 것이 중요하다.   
CAM 모델이 __특정 부위에 집중해 학습하는 것을 막기 위해 ACoL은 브랜치를 두 가지로 나누어 너무 높은 점수를 지워 주변의 특성 또한 반영__하도록 했다. 이 __과정을 논문에서 Adversial, 즉 적대적인 학습방법__이라고 표현한다.

ACoL은 여기서 얻어진 활성화 맵을 적대적인 방법으로 사용한다. 일정 값 이상 활성화된 맵을 지우도록 하는 것이다. `Classifier A`는 쉽게 전체적인 이미지를 보고 클래스를 판별할 수 있는데, `Classfier B`는 A의 CAM에서 크게 활성화된 영역을 지운 활성화 맵에서 분류를 해야 하기 때문에 더 어려운 문제를 푸는 것으로 볼 수 있다.

이렇게 `Classifier A, B`를 학습시켜 더 넓은 영역을 판별의 근거로 삼을 수 있다. 이 과정을 통해 모델은 쉽게 맞출 수 있는 샘플을 어렵게 다시 한 번 학습하는 과정을 거친다.

결과적으로 기존 CAM이 활성화되는 부분과 달리 ACoL은 물체의 전반적인 영역으로 CAM이 활성화되는 효과를 확인할 수 있다.

### 3) 1x1 Conv
![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-3-L-15.max-800x600.png)

CAM에서 CAM을 얻기 위해 대상이 되는 네트워크에 feed forward를 하고 활성화 맵에 가중치 계산을 다시 해야한다. 이 과정은 관찰하고자 하는 분류 모델의 feed forward와 별개의 작업으로, 물체 검출을 위한 모델로 사용하기 위해 모델의 feed forward 외 별도의 연산을 해주는 단점이 있다.

ACoL 논문은 이를 개선하기 위해 __필터 사이즈 1x1, 출력 채널의 개수는 분류하고자 하는 클래스 개수__를 가진 conv layer를 특성 맵에 사용하고 여기에 GAP을 적용하여 Network in Network에서 본 구조와 유사한 방식을 사용하고 있다.   
여기서 conv layer의 출력값은 바로 활성화 맵이 된다. 이렇게 구해진 활성화 맵과 CAM의 비교 결과는 위 그림에서 볼 수 있다.

# 5-6. 생각해보기

이렇게 분류 모델을 사용할 수 있고, 활성화 맵을 통해 모델이 어느 영역을 보고 답을 내리는지도 알 수 있게 되었다.

CAM을 활용하면 멋진 프로젝트들을 만들 수 있다.